### Experiments From "Filtered Feelings: Investigating Frequency Filters in Speech Emotion Recognition Models"
Created by: Teun van Gisteren (s1055104)

### Import libraries

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import os
import csv
import re
from scipy.cluster import hierarchy
import numpy as np

# 5.8 Visualisation Methods
## Confusion Matrices

In [ ]:
results_location = None # Where the results .csv files are

In [ ]:
# Function to generate a confusion matrix for results, with an optional session id for titling purposes
def generate_confusion_matrix(results, session=0, low_bound=-1, high_bound=-1):

    # Retrieve true and predicted labels from list of dictionaries
    actual = [item["Emotion"] for item in results]
    predicted = [item["Emotion_Guess"] for item in results]
    # Get all possible labels
    labels = set(actual).union(set(predicted))

    # Create confusion matrix and plot it
    confusion_matrix = metrics.confusion_matrix(actual, predicted)
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels = labels)
    cm_display.plot()

    # Add title to the plot
    title = f"Confusion Matrix of SpeechBrain on IEMOCAP "

    # Add frequency range info if applicable
    if low_bound != -1 and high_bound != -1:
        title += f"(l{low_bound}, h{high_bound}) "
        
    # If the session is 0, that means that it is the overall confusion matrix
    title += "data" if session == 0 else f"Session {session} data"
    plt.title(title)
    
    plt.show()

# Fucntion to calculate the accuracy of the results, with optional decimal number length parameter 
def calculate_accuracy_str(results, dec_len=10):
    actual = [item["Emotion"] for item in results]
    predicted = [item["Emotion_Guess"] for item in results]
    score = accuracy_score(actual, predicted)
    return f"Accuracy: {score * 100:.{dec_len}f}%"


In [ ]:
# Find all the .csv files in the specified results location
files = os.listdir(results_location)
files = [file for file in files if file.endswith(".csv")]

# Loop through each file in the directory
for filename in files:
    results = []
    # Open the CSV file
    with open(os.path.join(results_location, filename), mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        # Put .csv file back in correct format
        for row in reader:
            results.append({
                "Sentence": row["Sentence"],
                "Emotion": row["Emotion"],
                "Emotion_Guess": row["Emotion_Guess"]
            })

    # Find the frequency range from the file name
    frequency_bounds = re.findall(r'\d+', filename)

    # Show stats
    if len(frequency_bounds) > 0:
        low_bound = int(frequency_bounds[0])
        high_bound = int(frequency_bounds[1])
        generate_confusion_matrix(results, 0, low_bound, high_bound)
    else:
        generate_confusion_matrix(results, 0)
    print(calculate_accuracy_str(results, 2))


## Dendrograms

In [ ]:
# Plot a dendrogram based on the confusion matrix of the results with either a custom title or with a session number
def generate_dendrogram(session, results, title=""):
    actual = [item["Emotion"] for item in results]
    predicted = [item["Emotion_Guess"] for item in results]
    labels = list(set(actual).union(set(predicted)))
    
    confusion_matrix = metrics.confusion_matrix(actual, predicted)
    linkage_matrix = hierarchy.ward(confusion_matrix)

    plt.figure(figsize=(10, 6))
    dendrogram = hierarchy.dendrogram(linkage_matrix, labels=labels, orientation='top')

    if title == "":
        title = f"Dendrogram of SpeechBrain on IEMOCAP Session {session} data" if not session == 0 else f"Dendrogram of SpeechBrain on IEMOCAP data"
    plt.title(title)
    plt.xlabel('Emotions')
    plt.ylabel('Distance')
    plt.show()

In [ ]:
# Find all the .csv files in the specified results location
files = os.listdir(results_location)
files = [file for file in files if file.endswith(".csv")]

# Loop through each file in the directory
for filename in files:
    results = []
    # Open the CSV file
    with open(os.path.join(results_location, filename), mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        # Put .csv file back in correct format
        for row in reader:
            results.append({
                "Sentence": row["Sentence"],
                "Emotion": row["Emotion"],
                "Emotion_Guess": row["Emotion_Guess"]
            })

    # Find the frequency range from the file name
    frequency_bounds = re.findall(r'\d+', filename)
    
    # Show stats
    if len(frequency_bounds) > 0:
        low_bound = int(frequency_bounds[0])
        high_bound = int(frequency_bounds[1])
        generate_dendrogram(0, results, f"Dendrogram of SpeechBrain on IEMOCAP data ({low_bound}, {high_bound})")
    else:
        generate_dendrogram(0, results)

    # Show stats
    generate_dendrogram(0, results, f"Dendrogram of SpeechBrain on IEMOCAP data ({low_bound}, {high_bound})")
    print(calculate_accuracy_str(results, 2))

## Heat Maps

In [ ]:
def generate_heatmaps(results_path, dataset_name, emotions, emotion_dict={}):
    # Get the .csv files from the given path
    files = os.listdir(results_path)
    # Filter out only .csv files and the Base file
    files = [file for file in files if file.endswith(".csv") and not "Base" in file]
    
    # Setup the figure with correct amount of subplots
    fig, axs = plt.subplots(len(emotions), len(emotions)-1, figsize=(15, 15))
    axs = axs.flatten()

    #Generate all pairs of emotions
    emotion_pairs = [(x, y) for x in emotions for y in emotions if x is not y]
    
    counter = 0
    for pair in emotion_pairs:
        low_freqs = []
        high_freqs = []
        accuracies = []
        labels = []
        
        # Loop through each file in the directory
        for filename in files:
            # Get the lower and upper bound of the frequency range from the file name
            numbers = re.findall(r'\d+', filename)
            low_freq = int(numbers[0])
            high_freq = int(numbers[1])

            # Check to make sure we do not plot anything that is invalid
            if low_freq > high_freq:
                continue
            
            correct = 0
            wrong = 0
    
            # Open the CSV file
            with open(os.path.join(results_path, filename), mode='r', newline='', encoding='utf-8') as file:
                reader = csv.DictReader(file)
    
                # Count the correct and incorrect classifications
                for row in reader:
                    if row["Emotion"] == pair[0] and row["Emotion_Guess"] == pair[0]:
                        correct += 1
                    if row["Emotion"] == pair[0] and row["Emotion_Guess"] == pair[1]:
                        wrong += 1
            # Calculate the accuracy score
            accuracy = correct / (correct + wrong) if correct + wrong > 0 else 0
    
            # Append data for plotting
            low_freqs.append(low_freq)
            high_freqs.append(high_freq)
            accuracies.append(accuracy)
            labels.append(os.path.basename(filename))
    
        # Normalize accuracies for color mapping
        accuracies = np.array(accuracies)
        norm = plt.Normalize(vmin=accuracies.min(), vmax=accuracies.max())
        colors = plt.cm.RdYlGn(norm(accuracies))
    
        # Create scatter plot on the corresponding subplot
        axs[counter].scatter(np.asarray(low_freqs), np.asarray(high_freqs), c=colors, s=100)
        
        # Add labels and title to the subplot
        axs[counter].set_xlabel('Low Frequency Cutoff')
        axs[counter].set_ylabel('High Frequency Cutoff')

        # Set the title according to the given translations, or the original class names
        if emotion_dict != {}:
            axs[counter].set_title(f'{emotion_dict[pair[0]]} vs {emotion_dict[pair[1]]}')
        else:
            axs[counter].set_title(f'{pair[0]} vs {pair[1]}')

        #Set limit of x-axis, looks nicer
        axs[counter].set_xlim(right=4000)
        counter += 1

    # Add colour bar
    cax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
    sm = plt.cm.ScalarMappable(cmap='RdYlGn', norm=plt.Normalize(vmin=0, vmax=1))
    sm.set_array([])
    fig.colorbar(sm, cax=cax, label='Ratio of correct classifications to misclassifications')
    
    # Adjust layout manually
    fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.4, hspace=0.4)
    fig.suptitle(f"Results of The {dataset_name} Dataset", fontsize=20, fontweight='bold')
    plt.show()

In [ ]:
dataset_name = "IEMOCAP"
iemocap_location = None
emotions = ["hap", "neu", "ang", "sad"]
emotion_dict = {"hap": "Happy", "neu": "Neutral", "sad": "Sad", "ang": "Anger"}

generate_heatmaps(iemocap_location, dataset_name, emotions, emotion_dict)

In [ ]:
dataset_name = "MELD"
meld_results_location = None
emotions = ["hap", "neu", "ang", "sad"]
emotion_dict = {"hap": "Happy", "neu": "Neutral", "sad": "Sad", "ang": "Anger"}

generate_heatmaps(meld_results_location, dataset_name, emotions, emotion_dict)